# Tokenization

*Unfinished...*

There are many types of tokens from raw text: characters, words, subwords, sentences, etc.. In this note, we mainly focus on **Word Tokenization/ Subword Tokenization**. The main purpose of word tokenization is that we need to <u> convert raw text into data that can be processed by model </u>. 


Comparisons among different tokenization strategies for NLP models:

- character tokenization:
    - pros: easy; small vocabulary (low dimension)
    - cons: long sequence
- subword tokenization:
    - trade-off 🤗
- word tokenization:
    - pros: easy; short sequence
    - cons: large vocabulary (high dimension, out-of-vocabulary)

## Table of Contents

- [Pre-requirement Knowledge](#pre-requirement-knowledge)
    - [Character Encoding](#character-encoding)
    - [Unicode Normalization](#unicode-normalization)
- [Rule-based Tokenization](#rule-based-tokenization)
    - [Basic Rules](#1-basic-rules-whitespace-blankspace-or-self-defined-rules)
    - [Advanced Rules](#2-advanced-rules-penn-treebank-ptb-tweet-mwe-spacy)
- [Statistics-based Tokenization](#statistic-based-tokenization)
    - [Byte-Pair Encoding (BPE)](#byte-pair-encoding-bpe)
        - [Byte-level BPE (BBPE)](#byte-level-bpe-bbpe)
    - [WordPiece](#wordpiece)
    - [Unigram](#unigram)
    - [SentencePiece](#sentencepiece)

    

## Pre-requirement Knowledge

### Character Encoding

**Basic Concepts:**
- Character Set
- Coded Character Set: convert Character into Code Point (numerical value that maps to a specific character). 
    - the Unicode Standard, ISO/IEC 8859-1 (Latin-1), Windows Code Page 1252
- Character Encoding Form: convert Code Point into Code Unit (particular sequences of bits). 
    - fixed width: ASCII (7-bit), EASCII (8-bit), ISO 8859-1 (8-bit), Windows CP 1252 (8-bit),  UTF-32/USC-4 (32-bit)
    - variable width:  UTF-8, UTF-16
- Character Encoding Scheme: convert Code Unit into Byte. 
    - UTF-16 BE (big endian)

**Encoding Procedure:**

Character -> Code Point -> Code Unit -> Bytes 

`'A'` --(Unicode)-> `U+0041` --(UTF-16)-> `0x0041` --(UTF-16 LE)-> `0x41 0x00`

`'A'` --(ASCII)-> `0x41`

### Unicode Normalization

**Problems:** two types of equivalence
- Canonical equivalence: different Unicode have same visual appearance
    - Combining sequence: Ç	↔	C+◌̧
    - Ordering of conbining marks: q+◌̇+◌̣	↔	q+◌̣+◌̇
    - Hangual & conjoining jamo: 가	↔	ᄀ +ᅡ
    - Singleton equivalence: Ω	↔	Ω
- Compatibility equivalence: same abstract chracter have distinct visual appearances
    - Font variants: ℌ	→	H
    - Linebreaking differences: [NBSP]	→	[SPACE]
    - Positional variant forms: ﻉ	→	‌ع‌
    - Circled variants: ①	→	1
    - Width variants: ｶ	→	カ
    - Rotated variants: ︷	→	{
    - Superscripts/subscripts: i⁹	→	i9
    - Squared characters: ㌀	→	アパート
    - Fractions: ¼	→	1/4
    - Other: ǆ	→	dž

**Solutions:** Unicode Normalization: determin whether any two Unicode strings are equivalent to each other.
- Normalization Form D (NFD): Canonical Decomposition. 
- Normalization Form C (NFC): Canonical Decomposition, followed by Canonical Composition
- Normalization Form KD (NFKD): Compatibility Decomposition
- Normalization Form KC (NFKC): Compatibility Decomposition, followed by Compatibility Composition

<img style="left;" src="assets/Norm.jpg" width="500">

References
- [Wikipedia](https://www.wikipedia.org/)
- [UNICODE NORMALIZATION FORMS](https://www.unicode.org/reports/tr15/)



## Rule-based Tokenization

tools: NLTK, spaCy, TextBlob, Gensim, AllenNLP, Keras, Transformers 

### 1. Basic Rules: whitespace, blankspace, or self-defined rules

In [6]:
from nltk.tokenize import  RegexpTokenizer, WhitespaceTokenizer, BlanklineTokenizer, WordPunctTokenizer
text = "A nice day, Let's do tokenization at U.K.!"
re_tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
ws_tokenizer = WhitespaceTokenizer() # the same as RegexpTokenizer(r'\s+', gaps=True)
bs_tokenizer = BlanklineTokenizer() # the same as RegexpTokenizer(r'\s*\n\s*\n\s*', gaps=True)
wp_tokenizer = WordPunctTokenizer() # the same as RegexpTokenizer(r'\w+|[^\w\s]+', gaps=True)
print(re_tokenizer.tokenize(text), " (RE)")
print(ws_tokenizer.tokenize(text), " (Whitespace)")
print(bs_tokenizer.tokenize(text), " (Blankline)")
print(wp_tokenizer.tokenize(text), " (Word+Punctuation)")

['A', 'nice', 'day', ',', 'Let', "'s", 'do', 'tokenization', 'at', 'U', '.K.!']  (RE)
['A', 'nice', 'day,', "Let's", 'do', 'tokenization', 'at', 'U.K.!']  (Whitespace)
["A nice day, Let's do tokenization at U.K.!"]  (Blankline)
['A', 'nice', 'day', ',', 'Let', "'", 's', 'do', 'tokenization', 'at', 'U', '.', 'K', '.!']  (Word+Punctuation)


### 2. Advanced Rules: Penn Treebank (PTB), Tweet, MWE, Spacy

In [8]:
# Penn Treebank (NLTK version)
from nltk.tokenize import NLTKWordTokenizer, TreebankWordTokenizer, TweetTokenizer, MWETokenizer
text = "A nice day, Let's do tokenization at the U.K.! :-D"
tb_tokenizer = TreebankWordTokenizer()
"""
The Treebank tokenizer uses regular expressions to tokenize text as in Penn Treebank, see utils/tokenizer.sed for reference
(1) split standard contractions, e.g. ``don't`` -> ``do n't`` and ``they'll`` -> ``they 'll``
(2) treat most punctuation characters as separate tokens
(3) split off commas and single quotes, when followed by whitespace
(4) separate periods that appear at the end of line
Reference: https://www.nltk.org/api/nltk.tokenize.TreebankWordTokenizer.html#nltk.tokenize.TreebankWordTokenizer
"""
w_tokenizer = NLTKWordTokenizer()
"""
The NLTK tokenizer that has improved upon the TreebankWordTokenizer with additional relues, they share the same algorithm
"""
print(w_tokenizer.tokenize(text), " (PTB)")
print(tb_tokenizer.tokenize(text), " (PTB+)")

# Special for Tweet
tweet_tokenizer = TweetTokenizer()
"""
tokenizer for Tweets, e.g emoj, Kaomoji
"""
print(tweet_tokenizer.tokenize(text), " (Tweet)")

# Merge multi-words
mwe_tokenizer = MWETokenizer()
mwe_tokenizer.add_mwe(('do', 'tokenization'))
"""
merge multi-word expressions into single tokens
"""
print(mwe_tokenizer.tokenize(tweet_tokenizer.tokenize(text)), " (MWE+Tweet)")

['A', 'nice', 'day', ',', 'Let', "'s", 'do', 'tokenization', 'at', 'the', 'U.K.', '!', ':', '-D']  (PTB)
['A', 'nice', 'day', ',', 'Let', "'s", 'do', 'tokenization', 'at', 'the', 'U.K.', '!', ':', '-D']  (PTB+)
['A', 'nice', 'day', ',', "Let's", 'do', 'tokenization', 'at', 'the', 'U', '.', 'K', '.', '!', ':-D']  (Tweet)
['A', 'nice', 'day', ',', "Let's", 'do_tokenization', 'at', 'the', 'U', '.', 'K', '.', '!', ':-D']  (MWE+Tweet)


In [10]:
# Spacy rules
import spacy
text = "A nice day, Let's do tokenization at U.K.!"
nlp = spacy.load('en_core_web_sm')
"""
(1) Does the substring match a tokenizer exception rule? For example, “don’t” does not contain whitespace, but should be split into two tokens, “do” and “n’t”, while “U.K.” should always remain one token.
(2) Can a prefix, suffix or infix be split off? For example punctuation like commas, periods, hyphens or quotes.
Reference: https://spacy.io/usage/spacy-101
"""
doc = nlp(text)
print([token.text for token in doc], " (spaCy)")

['A', 'nice', 'day', ',', 'Let', "'s", 'do', 'tokenization', 'at', 'U.K.', '!']  (spaCy)


## Statistic-based Tokenization

### Byte-Pair Encoding (BPE)

**Brief Introduction (data compression)**

> The algorithm compresses data by finding the most frequently occurring pairs of adjacent bytes in the data and replacing all instances of the pair with a byte that was not in the original data. The algorithm repeats this process until no further compression is possible, either because there are no more frequently occurring pairs or there are no more unused bytes to represent pairs. [1]

**Example (Wiki)**

Suppose the data to be encoded is `aaabdaaabac`, The byte pair "aa" occurs most often, so it will be replaced by a byte that is not used in the data, such as "Z". Now there is the following data and replacement table: `ZabdZabac`, `Z=aa`. Then the process is repeated with byte pair "ab", replacing it with "Y": `ZYdZYac`, `Y=ab, Z=aa`. The only literal byte pair left occurs only once, and the encoding might stop here. Alternatively, the process could continue with recursive byte pair encoding, replacing "ZY" with "X": `XdXac`, `X=ZY, Y=ab, Z=aa`. This data cannot be compressed further by byte pair encoding because there are no pairs of bytes that occur more than once.

**Word Segmentation Algorithm**
> Firstly, we initialize the symbol vocabulary with the character vocabulary, and represent each word as a sequence of characters, plus a special end-ofword symbol ‘·’, which allows us to restore the original tokenization after translation. We iteratively count all symbol pairs and replace each occurrence of the most frequent pair (‘A’, ‘B’) with a new symbol ‘AB’. Each merge operation produces a new symbol which represents a character n-gram. Frequent character n-grams (or whole words) are eventually merged into a single symbol, thus BPE requires no shortlist. The final symbol vocabulary size is equal to the size of the initial vocabulary, plus the number of merge operations – the latter is the only hyperparameter of the algorithm. [2]


*References*
> [1] Gage, Philip. “A new algorithm for data compression.” The C Users Journal archive 12 (1994): 23-38. 

> [2] Sennrich, Rico et al. “Neural Machine Translation of Rare Words with Subword Units.” ArXiv abs/1508.07909 (2015): n. pag.

**Pretrained Language Models**

GPT-2, RoBERTa, BART, DeBERTa

**Implementation**

huggingface-tokenizer training script: https://github.com/EleutherAI/gpt-neo/blob/master/data/train_tokenizer.py

In [19]:
# The following pipeline comes from the original paper: Neural Machine Translation of Rare Words with Subword Units.
import re, collections
vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>':6, 'w i d e s t </w>':3} # add special token </w> for detokenize
print("before BPE: ", vocab)
num_merges = 10 # evidence: 1) empirical testing; 2) computational constraints; 3) common choices; 4) frequency plateau
for i in range(num_merges):
    # step-1: get the pair frequence
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq 
    best = max(pairs, key=pairs.get)
    # step-2: merge most pairs
    v_out = {}
    bigram = re.escape(' '.join(best))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in vocab:
        w_out = p.sub(''.join(best), word)
        v_out[w_out] = vocab[word]
    vocab = v_out
print("after BPE: ", vocab)


before BPE:  {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}
after BPE:  {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [5]:
# The following pipeline is modified from tokenizer: https://github.com/huggingface/tokenizers (the original code is implemented by RUST)
from collections import defaultdict
# 0.0 normalization: some general cleanup, such as removing needless whitespace, lowercasing, and/or removing accents
raw_text = "lôw low low low low lowér Lower newest newest newest newest newest newest widest widest widest"
text = "low low low low low lower lower newest newest newest newest newest newest widest widest widest"
# 0.1 pre-tokenization: add special tag to the begining of each word
text = "low Ġlow Ġlow Ġlow Ġlow Ġlower Ġlower Ġnewest Ġnewest Ġnewest Ġnewest Ġnewest Ġnewest Ġwidest Ġwidest Ġwidest"
# 1. add all special tokens to the vocabulary
vocab = ['UNK']
# 2. compute the initial alphabet
vocab = ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', 'Ġ', 'UNK']
# 3. tokenize word: get the word frequency by rule-based tokenization strategy
word_freq = {'low' : 1, 'Ġlow': 4, 'Ġlower' : 2, 'Ġnewest':6, 'Ġwidest':3}
print("vocab before bpe: ", vocab)
# 4. core algorithm: rank and merge
vocab_size = 20
merges = {} # used for further tokenize
splits = {word: [c for c in word] for word in word_freq.keys()} # intermedia results 
while len(vocab) < vocab_size:
    # count pairs in words
    pair_freqs = defaultdict(int)
    for word, freq in word_freq.items():
        split = splits[word]
        if len(split) == 1: continue
        for i in range(len(split)-1):
            pair = (split[i], split[i+1])
            pair_freqs[pair] += freq
    # get most frequent pair
    best_pair = ''
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    # merge pair
    for word in word_freq:
        split = splits[word]
        if len(split) == 1: continue
        i = 0
        while i < len(split)-1:
            if split[i] == best_pair[0] and split[i+1] == best_pair[1]:
                split = split[:i] + [best_pair[0]+best_pair[1]] + split[i+2:]
            else:
                i += 1
        splits[word] = split
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0]+best_pair[1])
print("vocab after bpe: ", vocab)

vocab before bpe:  ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', 'Ġ', 'UNK']
vocab after bpe:  ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', 'Ġ', 'UNK', 'es', 'est', 'lo', 'low', 'Ġlow', 'Ġn', 'Ġne', 'Ġnew']


#### Byte-level BPE (BBPE)

**Brief Introduction**

the basic unit for BPE is byte

example[3]

<img style="left;" src="assets/BBPE.png" width="600">

*Reference*
> [3] Wang, Changhan et al. “Neural Machine Translation with Byte-Level Subwords.” ArXiv abs/1909.03341 (2019): n. pag.


**Pretrained Language Models**
GPT2, RoBERTa

### WordPiece

**Brief Introduction**
Difference with BPE: the merge creteria: Choose the new word unit out of all possible ones that increases the likelihood on the training data the most when added to the model [4]

**Pretrained Models**

Bert, DistilBert

*Reference*
> [4] Schuster, Mike and Kaisuke Nakajima. “Japanese and Korean voice search.” 2012 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP) (2012): 5149-5152.

> [5] Wu, Yonghui et al. “Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation.” ArXiv abs/1609.08144 (2016): n. pag.

**Implementation**

Huggingface's Version:

The main difference is the way the pair to be merged is selected. Instead of selecting the most frequent pair, WordPiece computes a score for each pair, using the following formula: `score=(freq_of_pair)/(freq_of_first_element×freq_of_second_element)`, see [WordPiece tokenization](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt) for details.



### Unigram

**Brief Introduction**
> Compared to BPE and WordPiece, Unigram works in the other direction: it starts from a big vocabulary and removes tokens from it until it reaches the desired vocabulary size.

*Reference*
> [6] Kudo, Taku. “Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates.” ArXiv abs/1804.10959 (2018): n. pag.

**Implementation**

Huggingface's Version:

[Unigram tokenization](https://hf-mirror.com/learn/nlp-course/chapter6/7?fw=pt)




### Sentencepiece

**Brief Introduction**
> Treats the input as a raw input stream (language unrelated)

**Pretrained Language Model**

Albert, XLNet, Marian, T5

*Reference*
> [7] Kudo, Taku and John Richardson. “SentencePiece: A simple and language independent subword tokenizer and detokenizer for Neural Text Processing.” Conference on Empirical Methods in Natural Language Processing (2018).


## Tools

- [minbpe](https://github.com/karpathy/minbpe), Minimal, clean code for the (byte-level) Byte Pair Encoding (BPE) algorithm commonly used in LLM tokenization. The BPE algorithm is "byte-level" because it runs on UTF-8 encoded strings.